In [4]:
import os
import numpy as np
from conditional_rate_matching import results_path
import ast
import matplotlib.pyplot as plt

def extract_data(directory, file):
    
    dicts_fashion_uniform = []
    dicts_fashion_OTl2 = []
    dicts_fashion_OTlog = []

    dicts_noise_uniform = []
    dicts_noise_OTl2 = []
    dicts_noise_OTlog = []

    dicts_emnist_uniform = []
    dicts_emnist_OTl2 = []
    dicts_emnist_OTlog = []

    for sub_dir in os.listdir(directory):
        file_path = os.path.join(directory, sub_dir, file)
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                dict_str = f.read()
                d = ast.literal_eval(dict_str)
                run = sub_dir.split('_')
                d['source']=run[0]
                d['target']=run[2]
                for i, k in enumerate(run):
                    if 'gamma' in k: d['gamma'] = float(run[i+1])
                    if 'max' in k: d['max'] = float(run[i+1])
                    if 'Thermos' in k: 
                        d['Thermostat'] = k.split('Thermos')[0]

                d['run'] = sub_dir

                if d['source']=='fashion':
                    if 'OTl2' in sub_dir: dicts_fashion_OTl2.append(d)
                    elif 'OTlog' in sub_dir: dicts_fashion_OTlog.append(d)
                    else: dicts_fashion_uniform.append(d)
                if d['source']=='noise':
                    if 'OTl2' in sub_dir: dicts_noise_OTl2.append(d)
                    elif 'OTlog' in sub_dir: dicts_noise_OTlog.append(d)
                    else: dicts_noise_uniform.append(d)
                if d['source']=='emnist':
                    if 'OTl2' in sub_dir: dicts_emnist_OTl2.append(d)
                    elif 'OTlog' in sub_dir: dicts_emnist_OTlog.append(d)
                    else: dicts_emnist_uniform.append(d)
            
        else:
            print('file not found in {}'.format(file_path))
    return ((dicts_noise_uniform, dicts_noise_OTl2, dicts_noise_OTlog), 
            (dicts_fashion_uniform, dicts_fashion_OTl2, dicts_fashion_OTlog),
            (dicts_emnist_uniform, dicts_emnist_OTl2, dicts_emnist_OTlog))

data = extract_data('/scratch/df630/conditional_rate_matching/results/crm/images/', 'metrics.txt')


file not found in /scratch/df630/conditional_rate_matching/results/crm/images/emnist_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.5__03h40s52_2024.05.22__36759169/metrics.txt
file not found in /scratch/df630/conditional_rate_matching/results/crm/images/emnist_to_mnist_unet_OTl2_coupling_ConstantThermostat_gamma_1.0__22h21s01_2024.05.21__36753222/metrics.txt
file not found in /scratch/df630/conditional_rate_matching/results/crm/images/fashion_to_mnist_unet_OTl2_coupling_ConstantThermostat_gamma_0.5__03h09s21_2024.05.22__36759165/metrics.txt
file not found in /scratch/df630/conditional_rate_matching/results/crm/images/fashion_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__22h49s00_2024.05.18__36689756/metrics.txt
file not found in /scratch/df630/conditional_rate_matching/results/crm/images/emnist_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__22h48s49_2024.05.18__36689760/metrics.txt
file not found in /scratch/df630/conditional_rate_matching/r

In [11]:

def get_gammas(gamma, source='noise', coupling='uniform'):
    if source=='noise': i=0
    elif source=='fashion': i=1
    elif source=='emnist': i=2
    if coupling=='uniform': k=0
    elif coupling=='OTl2': k=1
    elif coupling=='OTlog': k=2
    fld_2=[]
    for d in data[i][k]:
        g = d['gamma']
        if g==gamma: 
            print(d['run'])
            fld_2.append(d['fid_2']) 

    fld = np.array(fld_2)
    print(fld)
    fld_mean = np.mean(fld)
    fld_std = np.std(fld)
    return fld_mean-fld_std, fld_mean, fld_mean+fld_std



def get_plot(source, fld, ax, label, title=('','',''),  xlim=[0, 2.5], ylim=[1,1000], color='blue', log=(False,False)):
    import pandas as pd
    import matplotlib.pyplot as plt

    ax[0].plot(fld['uniform'][source]['gamma'], fld['uniform'][source]['high'], color=color, lw=0.0)
    ax[0].plot(fld['uniform'][source]['gamma'], fld['uniform'][source]['center'], color=color, label=label, lw=0.5)
    ax[0].plot(fld['uniform'][source]['gamma'], fld['uniform'][source]['low'], color=color, lw=0.0)
    ax[0].fill_between(fld['uniform'][source]['gamma'], fld['uniform'][source]['high'],fld['uniform'][source]['low'],color=color,alpha=.2)    
    ax[0].set_ylabel('Frechet distance')
    ax[0].set_xlabel(r'$\beta$')
    ax[0].set_xlim(xlim)
    ax[0].set_ylim(ylim)
    ax[0].grid(linestyle='--')
    ax[0].legend(fontsize=7,loc='upper center')
    # ax[0].set_yticks([0, 200, 400, 600, 800,1000])
    ax[0].set_title('uniform coupling', fontsize=10)
    if log[0]: ax[0].set_xscale('log')
    if log[1]: ax[0].set_yscale('log')

    ax[1].plot(fld['OTlog'][source]['gamma'], fld['OTlog'][source]['high'], color=color, lw=0.)
    ax[1].plot(fld['OTlog'][source]['gamma'], fld['OTlog'][source]['center'], color=color, label=label, lw=0.5)
    ax[1].plot(fld['OTlog'][source]['gamma'], fld['OTlog'][source]['low'], color=color, lw=0.)
    ax[1].fill_between(fld['OTlog'][source]['gamma'], fld['OTlog'][source]['high'],fld['OTlog'][source]['low'],color=color,alpha=.2)    
    ax[1].set_ylabel('')
    ax[1].set_xlabel(r'$\beta$')
    ax[1].set_xlim(xlim)
    ax[1].set_ylim(ylim)
    ax[1].grid(linestyle='--')
    ax[1].legend(fontsize=7,loc='upper center')
    ax[1].set_title('OT Log-coupling', fontsize=10)
    if log[0]: ax[1].set_xscale('log')
    if log[1]: ax[1].set_yscale('log')

    ax[2].plot(fld['OTl2'][source]['gamma'], fld['OTl2'][source]['high'], color=color, lw=0.)
    ax[2].plot(fld['OTl2'][source]['gamma'], fld['OTl2'][source]['center'], color=color, label=label, lw=0.5)
    ax[2].plot(fld['OTl2'][source]['gamma'], fld['OTl2'][source]['low'], color=color, lw=0.)
    ax[2].fill_between(fld['OTl2'][source]['gamma'], fld['OTl2'][source]['high'],fld['OTl2'][source]['low'],color=color,alpha=.2)    
    ax[2].set_ylabel('')
    ax[2].set_xlabel(r'$\beta$')
    ax[2].set_xlim(xlim)
    ax[2].set_ylim(ylim)
    ax[2].legend(fontsize=7,loc='upper center')
    ax[2].grid(linestyle='--')
    ax[2].set_title(r'OT $\ell^2$-coupling', fontsize=10)
    if log[0]: ax[2].set_xscale('log')
    if log[1]: ax[2].set_yscale('log')


def get_fld(source, coupling):
    gammas = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.25, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
    fld = {}
    fld['low'] = []
    fld['high']  = []
    fld['center'] = []
    fld['gamma'] = []
    for g in gammas:
        low, center, high= get_gammas(gamma=g, source=source, coupling=coupling)
        fld['gamma'].append(g)
        fld['low'].append(low)
        fld['center'].append(center)
        fld['high'].append(high)
    return fld

FLD = {}
FLD['uniform'] = {}
FLD['OTl2'] = {}
FLD['OTlog'] = {}

FLD['uniform']['noise'] = get_fld(source='noise', coupling='uniform')
FLD['uniform']['fashion'] = get_fld(source='fashion', coupling='uniform')
FLD['uniform']['emnist'] = get_fld(source='emnist', coupling='uniform')
FLD['OTlog']['noise'] = get_fld(source='noise', coupling='OTlog')
FLD['OTlog']['fashion'] = get_fld(source='fashion', coupling='OTlog')
FLD['OTlog']['emnist'] = get_fld(source='emnist', coupling='OTlog')
FLD['OTl2']['noise'] = get_fld(source='noise', coupling='OTl2')
FLD['OTl2']['fashion'] = get_fld(source='fashion', coupling='OTl2')
FLD['OTl2']['emnist'] = get_fld(source='emnist', coupling='OTl2')

noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__22h49s00_2024.05.18__36689758
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__04h35s03_2024.05.20__36714363
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__19h18s26_2024.05.18__36687531
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__01h31s12_2024.05.20__36714157
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__17h54s10_2024.05.19__36700681
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.001__06h35s33_2024.05.20__36715629
[677.66815186 783.46893311 519.38452148 450.97598267 431.71643066
 657.15606689]
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.005__22h49s00_2024.05.18__36689759
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.005__19h18s26_2024.05.18__36687532
noise_to_mnist_unet_uniform_coupling_ConstantThermostat_gamma_0.005__04h35s38_2024.05.20__36714364
noise_to_mnist_unet_uniform_

In [6]:

fig, ax = plt.subplots(1, 3, figsize=(10, 2.5))
get_plot('noise', FLD, ax, label=r'noise $\to$ digits', color='blue',  xlim=[0,2], ylim=(1,1e6),  log=(False,True))
get_plot('fashion', FLD, ax, label=r'fashion $\to$ digits', color='green',  xlim=[0,2],ylim=(1,1e6),  log=(False,True))
get_plot('emnist', FLD, ax, label=r'letters $\to$ digits', color='red',  xlim=[0,2],ylim=(1,1e6),  log=(False,True))
plt.tight_layout()
plt.show()


In [3]:


fig, ax = plt.subplots(1, 3, figsize=(10, 2.5))
get_plot('noise', FLD, ax, label=r'noise $\to$ digits', color='blue',  xlim=[0,2], ylim=(1,1e6),  log=(False,True))
get_plot('fashion', FLD, ax, label=r'fashion $\to$ digits', color='green',  xlim=[0,2],ylim=(1,1e6),  log=(False,True))
get_plot('emnist', FLD, ax, label=r'letters $\to$ digits', color='red',  xlim=[0,2],ylim=(1,1e6),  log=(False,True))
plt.tight_layout()
plt.show()


In [7]:
FLD['uniform']['noise']['center']
FLD['OTlog']['noise']['center']
FLD['OTl2']['noise']['center']

[nan,
 nan,
 165.16303253173828,
 84.91205978393555,
 72.40573501586914,
 47.00967597961426,
 61.395225524902344,
 28.75601100921631,
 37.57185649871826,
 21.275885105133057,
 24.892712593078613,
 26.858150482177734,
 24.142168045043945,
 20.95871925354004,
 29.195587158203125,
 38.09352493286133,
 109.79510498046875,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [ ]:

def plot_loss(loss, ax, color='blue', log=(False,False), figsize=(5, 2.5)):
    _, ax = plt.subplots(1, 1, figsize=figsize)
    ax.plot(loss, color=color, lw=0.0)
    ax.set_ylabel('Frechet distance')
    ax.set_xlabel(r'$\beta$')
    ax.grid(linestyle='--')
    ax.legend(fontsize=7,loc='upper center')
    ax.set_title('uniform coupling', fontsize=10)
    if log[0]: ax.set_xscale('log')
    if log[1]: ax.set_yscale('log')
